In [ ]:
# default_exp model_fn


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Multitask Learning Model
Multitask Learning Model


In [ ]:
# export
from typing import Dict, Tuple
from inspect import signature
from copy import copy

from loguru import logger
import tensorflow as tf
from tensorflow.python.util.nest import flatten_with_joined_string_paths
import transformers

from m3tl.modeling import MultiModalBertModel
from m3tl.params import Params
from m3tl.utils import (get_embedding_table_from_model, get_transformer_main_model, create_dict_from_nested_model, variable_summaries, get_phase, set_phase)
from m3tl.special_tokens import TRAIN, EVAL, PREDICT


@tf.function
def filter_loss(loss, features, problem):

    if tf.reduce_mean(input_tensor=features['%s_loss_multiplier' % problem]) == 0:
        return_loss = 0.0
    else:
        return_loss = loss

    return return_loss


class BertMultiTaskBody(tf.keras.Model):
    """Model to extract bert features and dispatch corresponding rows to each problem_chunk.

    for each problem chunk, we extract corresponding features
    and hidden features for that problem. The reason behind this
    is to save computation for downstream processing.
    For example, we have a batch of two instances and they're from
    problem a and b respectively:
    Input:
    [{'input_ids': [1,2,3], 'a_loss_multiplier': 1, 'b_loss_multiplier': 0},
     {'input_ids': [4,5,6], 'a_loss_multiplier': 0, 'b_loss_multiplier': 1}]
    Output:
    {
      'a': {'input_ids': [1,2,3], 'a_loss_multiplier': 1, 'b_loss_multiplier': 0}
      'b': {'input_ids': [4,5,6], 'a_loss_multiplier': 0, 'b_loss_multiplier': 1}
    }

    """

    def __init__(self, params: Params, name='BertMultiTaskBody'):
        super(BertMultiTaskBody, self).__init__(name=name)
        self.params = params
        self.bert = MultiModalBertModel(params=self.params)
        if self.params.custom_pooled_hidden_size:
            self.custom_pooled_layer = tf.keras.layers.Dense(
                self.params.custom_pooled_hidden_size, activation=tf.keras.activations.selu)
        else:
            self.custom_pooled_layer = None

    # @tf.function
    def get_features_for_problem(self, features, hidden_feature, problem):
        # get features with ind == 1
        mode = get_phase()
        if mode == PREDICT:
            feature_this_round = features
            hidden_feature_this_round = hidden_feature
        else:
            multiplier_name = '%s_loss_multiplier' % problem

            # record_ind = tf.where(tf.cast(
            #     tf.squeeze(features[multiplier_name]), tf.bool))

            record_ind = tf.where(tf.cast(features[multiplier_name], tf.bool))

            hidden_feature_this_round = {}
            for hidden_feature_name in hidden_feature:
                if hidden_feature_name != 'embed_table':
                    hidden_feature_this_round[hidden_feature_name] = tf.squeeze(tf.gather(
                        hidden_feature[hidden_feature_name], record_ind, axis=0
                    ), axis=1)
                    hidden_feature_this_round[hidden_feature_name].set_shape(
                        [None, *hidden_feature[hidden_feature_name].shape.as_list()[1:]])
                else:
                    hidden_feature_this_round[hidden_feature_name] = hidden_feature[hidden_feature_name]

            feature_this_round = {}
            for features_name in features:
                feature_this_round[features_name] = tf.gather_nd(
                    features[features_name],
                    record_ind)

        return feature_this_round, hidden_feature_this_round

    def call(self, inputs: Dict[str, tf.Tensor]) -> Tuple[Dict[str, Dict[str, tf.Tensor]], Dict[str, Dict[str, tf.Tensor]]]:
        mode = get_phase()
        inputs = copy(inputs)
        # keys: ['last_hidden_state', 'pooler_output', 'hidden_states', 
        # 'attentions', 'model_input_mask', 'model_token_type_ids'
        # model_input_mask, model_token_type_ids]
        features, bert_model_outputs = self.bert(inputs, mode==TRAIN)

        # extract bert hidden features
        features['model_input_mask'] = bert_model_outputs['model_input_mask']
        features['model_token_type_ids'] = bert_model_outputs['model_token_type_ids']

        hidden_feature = {}
        for logit_type in ['seq', 'pooled', 'all', 'embed', 'embed_table']:
            if logit_type == 'seq':
                # tensor, [batch_size, seq_length, hidden_size]
                hidden_feature[logit_type] = bert_model_outputs['last_hidden_state']
            elif logit_type == 'pooled':
                # tensor, [batch_size, hidden_size]
                hidden_feature[logit_type] = bert_model_outputs['pooler_output']
                if self.custom_pooled_layer:
                    hidden_feature[logit_type] = self.custom_pooled_layer(
                        hidden_feature[logit_type])
            elif logit_type == 'all':
                # list, num_hidden_layers * [batch_size, seq_length, hidden_size]
                hidden_feature[logit_type] = bert_model_outputs['all_encoder_layers']
            elif logit_type == 'embed':
                # for res connection
                hidden_feature[logit_type] = bert_model_outputs['embedding_output']
            elif logit_type == 'embed_table':
                hidden_feature[logit_type] = bert_model_outputs['embedding_table']

        if self.params.detail_log:
            weight_var: tf.Variable
            for weight_var in self.bert.weights:
                variable_summaries(weight_var, 'body')

        # for each problem chunk, we extract corresponding features
        # and hidden features for that problem. The reason behind this
        # is to save computation for downstream processing.
        # For example, we have a batch of two instances and they're from
        # problem a and b respectively:
        # Input:
        # [{'input_ids': [1,2,3], 'a_loss_multiplier': 1, 'b_loss_multiplier': 0},
        #  {'input_ids': [4,5,6], 'a_loss_multiplier': 0, 'b_loss_multiplier': 1}]
        # Output:
        # {
        #   'a': {'input_ids': [1,2,3], 'a_loss_multiplier': 1, 'b_loss_multiplier': 0}
        #   'b': {'input_ids': [4,5,6], 'a_loss_multiplier': 0, 'b_loss_multiplier': 1}
        # }
        # NOTE: This logic now move to mtl model
        return_feature = {}
        return_hidden_feature = {}
        return_feature['all'] = features
        return_hidden_feature['all'] = hidden_feature
        return return_feature, return_hidden_feature

In [ ]:
# hide
from m3tl.test_base import TestBase
import m3tl
import shutil
import numpy as np
test_base = TestBase()
test_base.params.assign_problem(
    'weibo_fake_ner&weibo_fake_cls|weibo_fake_multi_cls|weibo_masklm')
params = test_base.params
train_dataset = m3tl.train_eval_input_fn(
    params=params, mode=m3tl.TRAIN)
eval_dataset = m3tl.train_eval_input_fn(
    params=params, mode=m3tl.EVAL
)

train_dataset = train_dataset.repeat()

one_batch_data = next(train_dataset.as_numpy_iterator())

2021-06-12 22:40:39.049 | INFO     | m3tl.base_params:register_multiple_problems:526 - Adding new problem weibo_fake_ner, problem type: seq_tag
2021-06-12 22:40:39.050 | INFO     | m3tl.base_params:register_multiple_problems:526 - Adding new problem weibo_fake_multi_cls, problem type: multi_cls
2021-06-12 22:40:39.051 | INFO     | m3tl.base_params:register_multiple_problems:526 - Adding new problem weibo_fake_cls, problem type: cls
2021-06-12 22:40:39.051 | INFO     | m3tl.base_params:register_multiple_problems:526 - Adding new problem weibo_masklm, problem type: masklm
2021-06-12 22:40:39.051 | INFO     | m3tl.base_params:register_multiple_problems:526 - Adding new problem weibo_fake_regression, problem type: regression
2021-06-12 22:40:39.052 | INFO     | m3tl.base_params:register_multiple_problems:526 - Adding new problem weibo_fake_vector_fit, problem type: vector_fit
2021-06-12 22:40:39.052 | INFO     | m3tl.base_params:register_multiple_problems:526 - Adding new problem weibo_pre

In [ ]:
mtl_body = BertMultiTaskBody(params=params)
set_phase(TRAIN)
features, hidden_features = mtl_body(one_batch_data)


404 Client Error: Not Found for url: https://huggingface.co/voidful/albert_chinese_tiny/resolve/main/tf_model.h5
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was tr

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


In [ ]:
# export 
class BertMultiTaskTop(tf.keras.Model):
    """Model to create top layer, aka classification layer, for each problem.
    """

    def __init__(self, params: Params, name='BertMultiTaskTop', input_embeddings: tf.Tensor = None):
        super(BertMultiTaskTop, self).__init__(name=name)
        self.params = params

        problem_type_layer = self.params.top_layer
        self.top_layer_dict = {}
        for problem_dict in self.params.run_problem_list:
            for problem in problem_dict:
                problem_type = self.params.problem_type[problem]
                # some layers has different signatures, assign inputs accordingly
                layer_signature_name = signature(
                    problem_type_layer[problem_type].__init__).parameters.keys()
                inputs_kwargs = {
                    'params': self.params,
                    'problem_name': problem
                }
                for signature_name in layer_signature_name:
                    if signature_name == 'input_embeddings':
                        inputs_kwargs.update(
                            {signature_name: input_embeddings})

                self.top_layer_dict[problem] = problem_type_layer[problem_type](
                    **inputs_kwargs)

    def call(self,
             inputs: Tuple[Dict[str, Dict[str, tf.Tensor]], Dict[str, Dict[str, tf.Tensor]]]) -> Dict[str, tf.Tensor]:
        inputs = copy(inputs)
        features, hidden_feature = inputs
        return_dict = {}

        mode = get_phase()

        for problem_dict in self.params.run_problem_list:
            for problem in problem_dict:
                feature_this_round = features[problem]
                hidden_feature_this_round = hidden_feature[problem]
                problem_type = self.params.problem_type[problem]

                # if pretrain, return pretrain logit
                if problem_type == 'pretrain':
                    pretrain = self.top_layer_dict[problem]
                    return_dict[problem] = pretrain(
                        (feature_this_round, hidden_feature_this_round))
                    return return_dict

                with tf.name_scope(problem):
                    layer = self.top_layer_dict[problem]

                    return_dict[problem] = layer(
                        (feature_this_round, hidden_feature_this_round))

        return return_dict

In [ ]:
# use embedding table as the classification top of mask lm
# top layer takes per-problem features and hidden features
from m3tl.utils import dispatch_features

features_per_problem, hidden_features_per_problem = {}, {}
for problem in params.problem_list:
    features_per_problem[problem], hidden_features_per_problem[problem] = dispatch_features(
        features=features['all'], hidden_feature=hidden_features['all'], problem=problem, mode=TRAIN
    )

input_embeddings = m3tl.utils.get_embedding_table_from_model(
    mtl_body.bert.bert_model)
mtl_top = BertMultiTaskTop(params=params, input_embeddings=input_embeddings)
set_phase(TRAIN)
logit_dict = mtl_top((features_per_problem, hidden_features_per_problem))


2021-06-12 22:40:58.059 | WARNING  | m3tl.problem_types.masklm:__init__:41 - Share embedding is enabled but hidden_size != embedding_size


In [ ]:
for problem, problem_logit in logit_dict.items():
    # last dim of logits equals to num_classes
    assert problem_logit.shape[-1] == params.get_problem_info(problem=problem, info_name='num_classes')

In [ ]:
# export 

class BertMultiTask(tf.keras.Model):
    def __init__(self, params: Params, name='BertMultiTask') -> None:
        super(BertMultiTask, self).__init__(name=name)
        self.params = params
        # initialize body model, aka transformers
        self.body = BertMultiTaskBody(params=self.params)

        self.mtl_model = self.params.mtl_model['model'](
            self.params, self.params.mtl_model['model_name'])
        self.mtl_model_include_top = self.params.mtl_model['include_top']

        # mlm might need word embedding from bert, map_structure
        # build sub-model
        input_embeddings = get_embedding_table_from_model(self.body.bert.bert_model)
        self.top = BertMultiTaskTop(
            params=self.params, input_embeddings=input_embeddings)

        # loss combination strategy
        self.loss_combination = self.params.loss_combination_strategy['model'](
            self.params, self.params.loss_combination_strategy['name'])
        self.eval_loss_metric_dict_list = []
        self.eval_metric_dict_list = []
        
        # for horovod
        self.first_batch = True

    def call(self, inputs):
        mode = get_phase()
        inputs = copy(inputs)
        pred_dict = {}
        body_outputs = self.body(
            inputs)
        if self.params.output_body_seq_hidden:
            pred_dict['seq'] = body_outputs[1]['all']['seq']
            
        if self.params.output_body_pooled_hidden:
            pred_dict['pooled'] = body_outputs[1]['all']['pooled']
            
        if self.mtl_model_include_top:
            problem_pred = self.mtl_model(
                body_outputs)
            pred_dict.update(problem_pred)
            return pred_dict
        
        mtl_model_outputs = self.mtl_model(
            body_outputs)
        
        if self.params.output_mtl_model_hidden:
            pred_dict['mtl'] = mtl_model_outputs
        
        pred_per_problem = self.top(
            (mtl_model_outputs))
        
        if self.params.output_problem_pred:
            pred_dict.update(pred_per_problem)
        return pred_dict

    def compile(self, *args, **kwargs):
        super(BertMultiTask, self).compile(*args, **kwargs)
        logger.critical('Initial lr: {}'.format(self.params.lr))
        logger.critical('Train steps: {}'.format(self.params.train_steps))
        logger.critical('Warmup steps: {}'.format(self.params.num_warmup_steps))
        self.optimizer, self.lr_scheduler = transformers.optimization_tf.create_optimizer(
            init_lr=self.params.lr,
            num_train_steps=self.params.train_steps,
            num_warmup_steps=self.params.num_warmup_steps,
            weight_decay_rate=0.01
        )
        self.mean_acc = tf.keras.metrics.Mean(name='mean_acc')

    def add_flatten_losses_metrics(self, return_dict: dict):
        current_eval_loss_dict = create_dict_from_nested_model(self, ele_name='losses')
        flatten_losses = flatten_with_joined_string_paths(current_eval_loss_dict)
        flatten_losses = {p: v for p, v in flatten_losses}
        return_dict.update(flatten_losses)
        return return_dict

    def train_step(self, data):
        set_phase(TRAIN)
        with tf.GradientTape() as tape:
            # Forward pass
            _ = self(data)
            # gather losses from all problems
            metric_dict = {m.name: m.result() for m in self.metrics}
            metric_dict = self.add_flatten_losses_metrics(return_dict=metric_dict)

            # apply loss combination strategy
            current_round_nest_loss_metric = create_dict_from_nested_model(self, ele_name='losses')
            combined_losses = self.loss_combination(
                current_loss_dict=current_round_nest_loss_metric,
                current_metric_dict=self.metrics,
                history=self.history
            )
            
        if self.params.use_horovod:
            import horovod.tensorflow as hvd
            
            tape = hvd.DistributedGradientTape(tape)
        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(combined_losses, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        self.mean_acc.update_state(
            [v for n, v in metric_dict.items() if n != 'mean_acc'])

        return_dict = metric_dict
        return_dict[self.mean_acc.name] = self.mean_acc.result()
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        
        if self.first_batch and self.params.use_horovod:
            self.first_batch = False
            hvd.broadcast_variables(trainable_vars, root_rank=0)
            hvd.broadcast_variables(self.optimizer.variables(), root_rank=0)
            
        return return_dict

    def test_step(self, data):
        """The logic for one evaluation step.

        This method can be overridden to support custom evaluation logic.
        This method is called by `Model.make_test_function`.

        This function should contain the mathemetical logic for one step of
        evaluation.
        This typically includes the forward pass, loss calculation, and metrics
        updates.

        Configuration details for *how* this logic is run (e.g. `tf.function` and
        `tf.distribute.Strategy` settings), should be left to
        `Model.make_test_function`, which can also be overridden.

        Arguments:
        data: A nested structure of `Tensor`s.

        Returns:
        A `dict` containing values that will be passed to
        `tf.keras.callbacks.CallbackList.on_train_batch_end`. Typically, the
        values of the `Model`'s metrics are returned.
        """
        set_phase(EVAL)
        y_pred = self(data)
        # Updates stateful loss metrics.
        self.compiled_loss(
            None, y_pred, None, regularization_losses=self.losses)

        self.compiled_metrics.update_state(None, y_pred, None)

        # get metrics to calculate mean
        m_list = []
        for metric in self.metrics:
            if 'mean_acc' in metric.name:
                continue
            if 'acc' in metric.name:
                m_list.append(metric.result())

            if 'f1' in metric.name:
                m_list.append(metric.result())

        self.mean_acc.update_state(
            m_list)

        ret_dict = {m.name: m.result() for m in self.metrics}
        ret_dict = self.add_flatten_losses_metrics(ret_dict)
        return ret_dict
        
    def predict_step(self, data):
        set_phase(PREDICT)
        return self(data)

In [ ]:
mtl = BertMultiTask(params=params)
logit_dict = mtl(one_batch_data)
for problem, problem_logit in logit_dict.items():
    # last dim of logits equals to num_classes
    assert problem_logit.shape[-1] == params.get_problem_info(problem=problem, info_name='num_classes')

404 Client Error: Not Found for url: https://huggingface.co/voidful/albert_chinese_tiny/resolve/main/tf_model.h5
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was tr

In [ ]:
mtl.compile()
hist = mtl.fit(train_dataset, validation_data=train_dataset, validation_steps=1, steps_per_epoch=1, epochs=3, verbose=1)

2021-06-12 22:41:04.946 | CRITICAL | __main__:compile:62 - Initial lr: 0.0
2021-06-12 22:41:04.947 | CRITICAL | __main__:compile:63 - Train steps: 0
2021-06-12 22:41:04.947 | CRITICAL | __main__:compile:64 - Warmup steps: 0


Epoch 1/3
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: invalid value for "node": expected "ast.AST", got "<class 'NoneType'>"; to visit lists of nodes, use "visit_block" instead
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1/1 [==============================] - ETA: 0s - mean_acc: 2.4794 - weibo_fake_cls_acc: 0.5000 - weibo_fake_ner_acc: 0.1143 - BertMultiTaskTop/weibo_fake_cls/losses/0: 0.9767 - BertMultiTaskTop/weibo_fake_multi_cls/losses/0: 1.2088 - BertMultiTaskTop/weibo_fake_ner/losses/0: 2.0114 - BertMultiTaskTop/weibo_masklm/losses/0: 10.0653

The parameters `output_attentions`, `output_hidden_states` and `use_cache` cannot be updated when calling a model.They have to be set to True/False in the config object (i.e.: `config=XConfig.from_pretrained('name', output_attentions=True)`).
The parameter `return_dict` cannot be set in graph mode and will always be set to `True`.


1/1 [==============================] - 14s 14s/step - mean_acc: 2.4794 - weibo_fake_cls_acc: 0.5000 - weibo_fake_ner_acc: 0.1143 - BertMultiTaskTop/weibo_fake_cls/losses/0: 0.9767 - BertMultiTaskTop/weibo_fake_multi_cls/losses/0: 1.2088 - BertMultiTaskTop/weibo_fake_ner/losses/0: 2.0114 - BertMultiTaskTop/weibo_masklm/losses/0: 10.0653 - val_loss: 14.3237 - val_mean_acc: 0.3214 - val_weibo_fake_cls_acc: 0.5000 - val_weibo_fake_ner_acc: 0.1429 - val_BertMultiTaskTop/weibo_fake_cls/losses/0: 0.9819 - val_BertMultiTaskTop/weibo_fake_multi_cls/losses/0: 1.2721 - val_BertMultiTaskTop/weibo_fake_ner/losses/0: 2.0120 - val_BertMultiTaskTop/weibo_masklm/losses/0: 10.0577
Epoch 2/3
1/1 [==============================] - 1s 717ms/step - mean_acc: 2.4846 - weibo_fake_cls_acc: 0.5000 - weibo_fake_ner_acc: 0.1143 - BertMultiTaskTop/weibo_fake_cls/losses/0: 0.9992 - BertMultiTaskTop/weibo_fake_multi_cls/losses/0: 1.1767 - BertMultiTaskTop/weibo_fake_ner/losses/0: 2.0816 - BertMultiTaskTop/weibo_mask

In [ ]:
params.output_body_pooled_hidden = True
params.output_body_seq_hidden = True
params.output_mtl_model_hidden = True
mtl = BertMultiTask(params=params)
logit_dict = mtl(one_batch_data)
assert 'pooled' in logit_dict
assert 'seq' in logit_dict
assert 'mtl' in logit_dict

404 Client Error: Not Found for url: https://huggingface.co/voidful/albert_chinese_tiny/resolve/main/tf_model.h5
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFAlbertModel: ['predictions.LayerNorm.bias', 'predictions.LayerNorm.weight', 'predictions.dense.weight', 'predictions.decoder.weight', 'predictions.dense.bias', 'predictions.decoder.bias', 'predictions.bias']
- This IS expected if you are initializing TFAlbertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFAlbertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFAlbertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was tr